In [1]:
import json
import os
import random
import csv
import re
import copy

In [2]:
def CSV2Json(c):
    heads = c[0]
    j = []
    for x in c[1:]:
        d = {}
        for i,y in enumerate(x):
            d[heads[i]]=y
        j.append(d)
    return j

In [3]:
with open("SubTaskA/Data/train_zero_shot.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    train_zero = list(gut)
    
#with open("SubtaskA/TestData/train_one_shot.csv", newline='',encoding='utf8') as csvfile:
#    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
#    train_one_test = list(gut)
    
with open("SubTaskA/Data/train_one_shot.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    train_one = list(gut)
    
with open("SubTaskA/Data/dev.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    devset = list(gut)
    
with open("SubTaskA/Data/dev_gold.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    devset_gold = list(gut)

with open("SubTaskA/Data/dev_submission_format.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    devset_sub = list(gut)
    
with open("SubTaskA/Data/eval.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    evalset = list(gut)
    
with open("SubTaskA/Data/eval_submission_format.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    evalset_sub = list(gut)
    
with open("SubTaskA/TestData/test_submission_format.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    testset_sub = list(gut)
    
with open("SubTaskA/TestData/test.csv", newline='',encoding='utf8') as csvfile:
    gut = csv.reader(csvfile, delimiter=',', quotechar='"')
    testset = list(gut)

In [4]:
csv_train_zero = CSV2Json(train_zero)
csv_train_one = CSV2Json(train_one)
#csv_train_one_dev_eval= CSV2Json(train_one_dev_eval)
csv_dev = CSV2Json(devset)
csv_devgold = CSV2Json(devset_gold)
csv_eval = CSV2Json(evalset)
csv_test = CSV2Json(testset)

In [5]:
semdev = []
semeval = []
semtest = []

In [6]:
semtrain0 = []
n = 0
er=0
for x in csv_train_zero:
    w = {"additionalInformation":{"context":{"nextSentence":x["Next"],"previousSentence":x["Previous"]}},"id":x["DataID"],"corpus":"SemevalZeroShot"}
    w["context"]=x["Target"]
    w["expression"]=x["MWE"]
    w["label"]=x["Label"]
    w["position"]=[]
    w["additionalInformation"]["language"]=x["Language"]
    w["additionalInformation"]["setting"]=x["Setting"]
    w["additionalInformation"]["semID"]=x["DataID"]
    w["additionalInformation"]['broadContext']=" ".join([x["Previous"],x["Target"],x["Next"]])
    patter = re.split(" |-",w["expression"].lower())
    patter = "[ -]".join(patter)
    sentence = w["context"].lower()
    paragraph = w["additionalInformation"]['broadContext'].lower()
    posi = []
    posi_long = []
    for match in re.finditer(patter, sentence):
        posi.append(match.span())
    for match in re.finditer(patter, paragraph):
        posi_long.append(match.span())
    if len(posi)==0:
        if w["expression"] == "efeito especial":
            patter = re.split(" |-","efeitos especiais")
            patter = "[ -]".join(patter)
            for match in re.finditer(patter, sentence):
                posi.append(match.span())
            for match in re.finditer(patter, paragraph):
                posi_long.append(match.span())
        elif w["expression"]=="círculo virtuoso":
            patter = re.split(" |-","circulo virtuoso")
            patter = "[ -]".join(patter)
            for match in re.finditer(patter, sentence):
                posi.append(match.span())
            for match in re.finditer(patter, paragraph):
                posi_long.append(match.span())
        elif w["expression"]=="paraíso fiscal":
            patter = re.split(" |-","paraísos fiscais")
            patter = "[ -]".join(patter)
            for match in re.finditer(patter, sentence):
                posi.append(match.span())
            for match in re.finditer(patter, paragraph):
                posi_long.append(match.span())
        if len(posi)==0:
            print(c,w["expression"],w["context"])
            er+=1     
        if len(posi_long)==0:
            print(c,"long",w["expression"],w["context"])
            er+=1     
    w["position"]=posi
    w["additionalInformation"]['positionBroadContext']=posi_long
    w["additionalInformation"]["firstPositionBroadContext"]=[posi[0][0]+len(x["Previous"])+1,posi[0][1]+len(x["Previous"])+1]
    w["dataSplit"]="train"
    w["expression_cased"]=w["additionalInformation"]["broadContext"][w["additionalInformation"]["firstPositionBroadContext"][0]:w["additionalInformation"]["firstPositionBroadContext"][1]]
    semtrain0.append(w)
    n+=1
    
        
print(er,len(semtrain0))

0 4491


In [7]:
semtrain1 = []
n = 0
er=0
for x in csv_train_one:
    w = {"additionalInformation":{"context":{"nextSentence":x["Next"],"previousSentence":x["Previous"]}},"id":x["DataID"],"corpus":"SemevalZeroShot"}
    w["context"]=x["Target"]
    w["expression"]=x["MWE"]
    w["label"]=x["Label"]
    w["additionalInformation"]["language"]=x["Language"]
    w["additionalInformation"]["setting"]=x["Setting"]
    w["additionalInformation"]["semID"]=x["DataID"]
    w["additionalInformation"]['broadContext']=" ".join([x["Previous"],x["Target"],x["Next"]])
    patter = re.split(" |-",w["expression"].lower())
    patter = "[ -]".join(patter)
    sentence = w["context"].lower()
    paragraph = w["additionalInformation"]['broadContext'].lower()
    posi = []
    posi_long = []
    for match in re.finditer(patter, paragraph):
        posi_long.append(match.span())
    for match in re.finditer(patter, sentence):
        posi.append(match.span())
    if len(posi)==0:
        print(c,w["expression"],w["context"])
        er+=1
    #posi = w["context"].index(w["expression"])
    #w["position"]=[posi,posi+len(w["expression"])]
    w["position"]=posi
    w["additionalInformation"]['positionBroadContext']=posi_long
    w["additionalInformation"]["firstPositionBroadContext"]=[posi[0][0]+len(x["Previous"])+1,posi[0][1]+len(x["Previous"])+1]
    w["dataSplit"]="train"
    w["expression_cased"]=w["additionalInformation"]["broadContext"][w["additionalInformation"]["firstPositionBroadContext"][0]:w["additionalInformation"]["firstPositionBroadContext"][1]]
    semtrain1.append(w)
    n+=1
    
        
print(er,len(semtrain1))

0 140


In [9]:
semdev = []

n = 0
er=0
for i,x in enumerate(csv_dev):
    w = {"additionalInformation":{"context":{"nextSentence":x["Next"],"previousSentence":x["Previous"]}},"id":csv_devgold[i]["DataID"],"corpus":"SemevalTaskA"}
    w["context"]=x["Target"]
    w["expression"]=x["MWE"]
    w["label"]=csv_devgold[i]["Label"]
    w["additionalInformation"]["semID"]=csv_devgold[i]["ID"]
    w["additionalInformation"]["language"]=csv_devgold[i]["Language"]
    w["additionalInformation"]['broadContext']=" ".join([x["Previous"],x["Target"],x["Next"]])
    w["dataSplit"]="dev"
    patter = re.split(" |-",w["expression"].lower())
    patter = "[ -]".join(patter)
    sentence = w["context"].lower()
    paragraph = w["additionalInformation"]['broadContext'].lower()
    posi = []
    posi_long = []
    for match in re.finditer(patter, sentence):
        posi.append(match.span())
    for match in re.finditer(patter, paragraph):
        posi_long.append(match.span())
    if len(posi)==0:
        print(c,w["expression"],w["context"])
        er+=1
    w["position"]=posi
    w["additionalInformation"]['positionBroadContext']=posi_long
    w["additionalInformation"]["firstPositionBroadContext"]=[posi[0][0]+len(x["Previous"])+1,posi[0][1]+len(x["Previous"])+1]
    w["dataSplit"]="dev"
    w["expression_cased"]=w["additionalInformation"]["broadContext"][w["additionalInformation"]["firstPositionBroadContext"][0]:w["additionalInformation"]["firstPositionBroadContext"][1]]
    semdev.append(w)
    n+=1
    
        
print(er,len(semdev))

0 739


In [10]:
semeval = []


n = 0
er=0
for x in csv_eval:
    w = {"additionalInformation":{"context":{"nextSentence":x["Next"],"previousSentence":x["Previous"]}},"id":"eval"+str(x["ID"]),"corpus":"SemevalTaskA"}
    w["context"]=x["Target"]
    w["expression"]=x["MWE"]
    w["additionalInformation"]["language"]=x["Language"]
    w["additionalInformation"]["semID"]=x["ID"]
    w["additionalInformation"]['broadContext']=" ".join([x["Previous"],x["Target"],x["Next"]])
    w["dataSplit"]="eval"
    patter = re.split(" |-",w["expression"].lower())
    patter = "[ -]".join(patter)
    sentence = w["context"].lower()
    paragraph =  w["additionalInformation"]['broadContext'].lower()
    posi,posi_long = [],[]
    for match in re.finditer(patter, sentence):
        posi.append(match.span())
    for match in re.finditer(patter, paragraph):
        posi_long.append(match.span())
    if len(posi)==0:
        if w["expression"] == "núcleo atômico":
            patter = re.split(" |-","núcleos atômicos")
            patter = "[ -]".join(patter)
            for match in re.finditer(patter, sentence):
                posi.append(match.span())      
            for match in re.finditer(patter, paragraph):
                posi_long.append(match.span())
        if len(posi)==0:
            print(c,w["expression"],w["context"])
            er+=1
    w["additionalInformation"]['positionBroadContext']=posi_long
    w["position"]=posi
    w["additionalInformation"]["firstPositionBroadContext"]=[posi[0][0]+len(x["Previous"])+1,posi[0][1]+len(x["Previous"])+1]
    w["dataSplit"]="eval"
    w["expression_cased"]=w["additionalInformation"]["broadContext"][w["additionalInformation"]["firstPositionBroadContext"][0]:w["additionalInformation"]["firstPositionBroadContext"][1]]
    semeval.append(w)
    n+=1
    if len(w["additionalInformation"]["positionBroadContext"])==0:
        print(posi, posi_long)
        
print(er,len(semeval))

0 762


In [11]:
semtest = []
#test ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']

n = 0
er=0
for x in csv_test:
    w = {"additionalInformation":{"context":{"nextSentence":x["Next"],"previousSentence":x["Previous"]}},"id":"test."+str(x["ID"]),"corpus":"SemevalTaskA"}
    w["context"]=x["Target"]
    w["expression"]=x["MWE"]
    w["additionalInformation"]["language"]=x["Language"]
    w["additionalInformation"]["semID"]=x["ID"]
    w["additionalInformation"]['broadContext']=" ".join([x["Previous"],x["Target"],x["Next"]])
    w["dataSplit"]="test"
    patter = re.split(" |-",w["expression"].lower())
    patter = "[ -]".join(patter)
    sentence = w["context"].lower()
    paragraph =w["additionalInformation"]['broadContext'].lower()
    posi,posi_long = [],[]
    for match in re.finditer(patter, sentence):
        posi.append(match.span())
    for match in re.finditer(patter, paragraph):
        posi_long.append(match.span())
    if len(posi_long)==0:
        print(w["expression"],w["paragraph"])
        er+=1
    w["additionalInformation"]['positionBroadContext']=posi_long
    w["additionalInformation"]["firstPositionBroadContext"]=[posi[0][0]+len(x["Previous"])+1,posi[0][1]+len(x["Previous"])+1]
    w["position"]=posi
    w["dataSplit"]="test"
    w["expression_cased"]=w["additionalInformation"]["broadContext"][w["additionalInformation"]["firstPositionBroadContext"][0]:w["additionalInformation"]["firstPositionBroadContext"][1]]
    semtest.append(w)
    n+=1
    
        
print(er,len(semtest))


0 2342


In [12]:
print(len(semtrain0))
print(len(semtrain1))
#len(semtrain1_dev_eval)
print(len(semdev))
print(len(semeval))
print(len(semtest))

4491
140
739
762
2342


In [13]:
#random.shuffle(semtrain0)
#random.shuffle(semtrain1)

In [14]:
semeval_all = {"semevalZeroShot":[],"semevalOneShot":[]}

for x in semtrain0:
    i = copy.deepcopy(x)
    i["corpus"]="semevalZeroShot"
    semeval_all["semevalZeroShot"].append(i)
    j = copy.deepcopy(x)
    j["corpus"]="semevalOneShot"
    semeval_all["semevalOneShot"].append(j)

    
for x in semtrain1:
    i = copy.deepcopy(x)
    i["corpus"]="semevalOneShot"
    semeval_all["semevalOneShot"].append(i)
    
for x in semdev:
    i = copy.deepcopy(x)
    i["corpus"]="semevalOneShot"
    semeval_all["semevalOneShot"].append(i)
    j = copy.deepcopy(x)
    j["corpus"]="semevalZeroShot"
    semeval_all["semevalZeroShot"].append(j)
  
for x in semeval:
    i = copy.deepcopy(x)
    i["corpus"]="semevalOneShot"
    semeval_all["semevalOneShot"].append(i)
    j = copy.deepcopy(x)
    j["corpus"]="semevalZeroShot"
    semeval_all["semevalZeroShot"].append(j)


for x in semtest:
    i = copy.deepcopy(x)
    i["corpus"]="semevalOneShot"
    semeval_all["semevalOneShot"].append(i)
    j = copy.deepcopy(x)
    j["corpus"]="semevalZeroShot"
    semeval_all["semevalZeroShot"].append(j)
    

In [15]:
random.shuffle(semeval_all["semevalZeroShot"])
random.shuffle(semeval_all["semevalOneShot"])

In [16]:
with open('semevalTaskA_shuffle.json', 'w', encoding='utf-8') as file:
    json.dump(semeval_all, file, indent='\t', ensure_ascii=False)